In [1]:
# Import AEMpy library
import aempy

### Connect and display the AEM version

In [2]:
# Connect to the AEM Instance
instance = aempy.AEM()

In [3]:
# Get the product information
productinfo = instance.info()

Request: http://localhost:4502/system/console/status-productinfo.json


In [4]:
# Print the product version
print(productinfo.version)

Adobe Experience Manager (6.5.5.0)


### Get the error logs

In [5]:
# Connect to the system console
system = aempy.System()

In [6]:
# Get the five first lines of the error.log file
errorlogs = system.log_error(5)

Request: http://localhost:4502/system/console/slinglog/tailer.txt?tail=5&grep=*&name=/logs/error.log


In [7]:
# Print the lines
print(errorlogs)

['26.08.2020 15:47:39.344 *INFO* [oak-repository-executor-1] com.adobe.granite.repository Service [8645, [org.apache.jackrabbit.oak.api.jmx.SessionMBean]] ServiceEvent REGISTERED', '26.08.2020 15:47:39.345 *INFO* [oak-repository-executor-1] com.adobe.granite.repository Service [8646, [org.apache.jackrabbit.oak.api.jmx.SessionMBean]] ServiceEvent REGISTERED', '26.08.2020 15:47:39.345 *INFO* [oak-repository-executor-1] com.adobe.granite.repository Service [8647, [org.apache.jackrabbit.oak.api.jmx.SessionMBean]] ServiceEvent REGISTERED', '26.08.2020 15:47:42.897 *ERROR* [sling-threadpool-67387070-8373-45bd-bead-d6282f6b4973-(apache-sling-job-thread-pool)-32-com_day_cq_replication_job_publish(com/day/cq/replication/job/publish)] com.day.cq.replication.impl.AgentManagerImpl Job contains agent which is not started: publish. waiting ReplicationJob{job=JobImpl [properties=org.apache.sling.api.wrappers.ValueMapDecorator@fcb7bbd0 : {event.job.retrydelay=60000, cq:type=DELETE, slingevent:applicat

### Parse the logs

In [8]:
# Let's get more line from error.log (default: 10.000 lines)
errorlogs = system.log_error()

Request: http://localhost:4502/system/console/slinglog/tailer.txt?tail=10000&grep=*&name=/logs/error.log


We import a new library named **pandas** (it has nothing to do with the animal). Pandas is a Data analysis and manipulation library, mainly using a dataframe object

In [9]:
# Import Pandas library
import pandas as pd

In [10]:
# Split the logs and arrange in a table using the Pandas library
dfLog = pd.DataFrame([sub.split(" ") for sub in errorlogs])

*Note: It is a good practice to have all the imports at the begining of your python program. But for the comprehension of this tutorial, we added it here.*

In [11]:
# Split the logs and arrange in a table using the Pandas library
dfLog = pd.DataFrame([sub.split(" ") for sub in errorlogs])

In [12]:
# Add Names to the columns
dfLog = dfLog.rename(columns={0:'date',1:'time',2:'level',3:'ID',4:'class',5:'msg'})

In [14]:
# Print the parsed logs
print(dfLog)

            date          time    level  \
0     26.08.2020  01:29:20.014   *INFO*   
1     26.08.2020  01:29:20.040   *INFO*   
2     26.08.2020  01:29:20.016   *INFO*   
3     26.08.2020  01:29:20.015   *INFO*   
4     26.08.2020  01:29:20.050   *INFO*   
...          ...           ...      ...   
2189  26.08.2020  15:47:39.344   *INFO*   
2190  26.08.2020  15:47:39.345   *INFO*   
2191  26.08.2020  15:47:39.345   *INFO*   
2192  26.08.2020  15:47:42.897  *ERROR*   
2193                      None     None   

                                                     ID  \
0                           [sling-default-5-Registered   
1                           [sling-default-3-Registered   
2     [sling-default-2-com.adobe.cq.wcm.translation....   
3                           [sling-default-1-Registered   
4     [sling-default-1-com.adobe.granite.threaddump....   
...                                                 ...   
2189                        [oak-repository-executor-1]   
2190       

### Convert logs to a pandas Dataframe

In [15]:
# There is a builtin parsing in AEMpy to automatically convert logs to pandas dataframe
dfErrors = system.log_to_pandas(errorlogs)

In [17]:
# Print the first 10 lines of the pandas dataframe
dfErrors.head()

,date,level,class,message
0,2020-08-26 01:29:20.014,INFO,sling-default-5-Registered,"[Service.1250], com.adobe.granite.taskmanageme..."
1,2020-08-26 01:29:20.040,INFO,sling-default-3-Registered,"[Service.3940], com.adobe.cq.wcm.jobs.async.im..."
2,2020-08-26 01:29:20.016,INFO,sling-default-2-com.adobe.cq.wcm.translation.i...,[com.adobe.cq.wcm.translation.impl.scheduler.S...
3,2020-08-26 01:29:20.015,INFO,sling-default-1-Registered,"[Service.1296], com.adobe.granite.oauth.server..."
4,2020-08-26 01:29:20.050,INFO,sling-default-1-com.adobe.granite.threaddump.i...,[com.adobe.granite.threaddump.impl.BackupClean...


In [18]:
# Ask pandas to provide a description of the dataframe
dfErrors.describe()

/Users/odyssee/opt/anaconda3/envs/aem/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  


,date,level,class,message
count,2193,2193,2193,2193
unique,2174,3,80,1667
top,2020-08-26 08:42:06.020000,INFO,Apache,[org.apache.sling.discovery.oak.SynchronizedCl...
freq,3,1773,640,416
first,2020-08-26 01:29:20.014000,NaN,NaN,NaN
last,2020-08-26 15:47:42.897000,NaN,NaN,NaN


### Analyze the logs with the *ERROR* level

In [20]:
# Filter the logs by level
dfLevelError = dfErrors[dfErrors["level"].str.match("ERROR")]

In [21]:
# Add a title for the next actions
print("Information about log errors:")

Information about log errors:


In [22]:
# Display the first 5 lines of the dataframe
dfLevelError.head()

,date,level,class,message
159,2020-08-26 07:45:05.851,ERROR,sling-threadpool-67387070-8373-45bd-bead-d6282...,"[com.day.cq.replication.impl.AgentManagerImpl,..."
165,2020-08-26 08:42:06.078,ERROR,sling-threadpool-67387070-8373-45bd-bead-d6282...,"[com.day.cq.replication.impl.AgentManagerImpl,..."
166,2020-08-26 08:50:27.257,ERROR,sling-threadpool-67387070-8373-45bd-bead-d6282...,"[com.day.cq.replication.impl.AgentManagerImpl,..."
168,2020-08-26 08:51:30.949,ERROR,sling-threadpool-67387070-8373-45bd-bead-d6282...,"[com.day.cq.replication.impl.AgentManagerImpl,..."
170,2020-08-26 08:53:38.216,ERROR,sling-threadpool-67387070-8373-45bd-bead-d6282...,"[com.day.cq.replication.impl.AgentManagerImpl,..."


In [23]:
# Display the last 5 lines of the dataframe
dfLevelError.tail()

,date,level,class,message
2178,2020-08-26 15:43:42.860,ERROR,sling-threadpool-67387070-8373-45bd-bead-d6282...,"[com.day.cq.replication.impl.AgentManagerImpl,..."
2180,2020-08-26 15:44:42.869,ERROR,sling-threadpool-67387070-8373-45bd-bead-d6282...,"[com.day.cq.replication.impl.AgentManagerImpl,..."
2182,2020-08-26 15:45:42.878,ERROR,sling-threadpool-67387070-8373-45bd-bead-d6282...,"[com.day.cq.replication.impl.AgentManagerImpl,..."
2185,2020-08-26 15:46:42.886,ERROR,sling-threadpool-67387070-8373-45bd-bead-d6282...,"[com.day.cq.replication.impl.AgentManagerImpl,..."
2192,2020-08-26 15:47:42.897,ERROR,sling-threadpool-67387070-8373-45bd-bead-d6282...,"[com.day.cq.replication.impl.AgentManagerImpl,..."


In [24]:
# Describe the table
dfLevelError.describe()

/Users/odyssee/opt/anaconda3/envs/aem/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  


,date,level,class,message
count,419,419,419,419
unique,419,1,35,419
top,2020-08-26 12:52:40.931000,ERROR,sling-threadpool-67387070-8373-45bd-bead-d6282...,"[com.day.cq.replication.impl.AgentManagerImpl,..."
freq,1,419,12,1
first,2020-08-26 07:45:05.851000,NaN,NaN,NaN
last,2020-08-26 15:47:42.897000,NaN,NaN,NaN
